In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predicta-1-0-predict-the-unpredictable-part-2/daily_data.csv
/kaggle/input/predicta-1-0-predict-the-unpredictable-part-2/submission.csv


In [2]:
daily_data=pd.read_csv('/kaggle/input/predicta-1-0-predict-the-unpredictable-part-2/daily_data.csv')
daily_data.info()
#daily_data.shape  ---> (2893, 17)

# Check for missing values
daily_data.isnull().sum()

daily_data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2893 entries, 0 to 2892
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   day_id                    2893 non-null   object 
 1   city_id                   2893 non-null   object 
 2   temperature_celsius       2893 non-null   float64
 3   condition_text            479 non-null    object 
 4   wind_kph                  2893 non-null   float64
 5   wind_degree               2893 non-null   int64  
 6   pressure_mb               2893 non-null   float64
 7   precip_mm                 2893 non-null   float64
 8   humidity                  2893 non-null   int64  
 9   cloud                     2893 non-null   int64  
 10  feels_like_celsius        2893 non-null   float64
 11  visibility_km             2893 non-null   float64
 12  uv_index                  2893 non-null   float64
 13  gust_kph                  2893 non-null   float64
 14  air_qual

,day_id,city_id,temperature_celsius,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,uv_index,gust_kph,air_quality_us-epa-index,sunrise,sunset
0,D0001,C001,27.0,NaN,6.1,210,1006.0,0.0,54,75,28.0,10.0,6.0,11.9,2,06:04 AM,07:19 PM
1,D0002,C001,22.0,NaN,6.1,170,1006.0,0.0,73,75,24.5,10.0,1.0,23.4,1,06:05 AM,07:18 PM
2,D0003,C001,20.0,Light Rain with Thunder,3.6,10,1011.0,4.5,100,75,20.0,10.0,1.0,12.6,1,06:05 AM,07:18 PM
3,D0004,C001,17.0,Clear and Sunny,6.1,150,1018.0,0.0,88,0,17.0,10.0,1.0,11.2,1,06:06 AM,07:16 PM
4,D0005,C001,18.0,NaN,3.6,92,1019.0,0.0,94,0,18.0,10.0,1.0,9.0,1,06:07 AM,07:15 PM
5,D0006,C001,20.0,NaN,3.6,96,1019.0,0.0,88,0,20.0,10.0,1.0,11.2,1,06:07 AM,07:13 PM
6,D0007,C001,21.0,Partly Cloudy,4.0,310,1015.0,0.0,100,50,21.0,10.0,1.0,15.1,2,06:08 AM,07:11 PM
7,D0008,C001,21.0,NaN,20.2,330,1011.0,0.0,53,75,21.0,10.0,1.0,17.3,1,06:09 AM,07:10 PM
8,D0009,C001,23.0,NaN,3.6,63,1013.0,0.0,47,75,24.8,10.0,1.0,20.5,1,06:10 AM,07:08 PM
9,D0010,C001,24.0,NaN,15.1,330,1014.0,0.0,54,75,25.3,10.0,1.0,9.7,1,06:11 AM,07:06 PM


We found that only 'condition_text' column has null values in the data set.

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [4]:
# Define the specified weather conditions
weather_conditions = ["Clear and Sunny", "Partly Cloudy", "Light Precipitation", "Cloudy and Overcast","Mist or Fog", "Rain Showers", "Light Rain with Thunder", "Thunderstorms", "Moderate to Heavy Rain"]

In [5]:
train_data = daily_data.dropna(subset=['condition_text'])

X = train_data.drop(columns=['day_id', 'city_id', 'condition_text', 'sunrise', 'sunset'])
y = train_data['condition_text']


In [6]:
# Encode the target variable 
label_encoder = LabelEncoder()
label_encoder.fit(weather_conditions)
y_encoded = label_encoder.transform(y)

X.fillna(X.mean(), inplace=True)

# Normalization
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Initialize the model
model = RandomForestClassifier(random_state=42)

# Hyperparameter tuning using GridSearchCV with cross-validation
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=9, n_jobs=-1, verbose=2)
grid_search.fit(X_scaled, y_encoded)


best_model = grid_search.best_estimator_

#cross-validation
cv_scores = cross_val_score(best_model, X_scaled, y_encoded, cv=9)

print("Cross-validation scores:", cv_scores)

print("Mean cross-validation score:", np.mean(cv_scores))


#Predict missing values in the original dataset
X_all = daily_data.drop(columns=['day_id', 'city_id', 'condition_text', 'sunrise', 'sunset'])
X_all_scaled = scaler.transform(X_all.fillna(X_all.mean()))
predictions = best_model.predict(X_all_scaled)


daily_data['condition_text'] = label_encoder.inverse_transform(predictions)

#Create the submission.csv file
submission = daily_data[['day_id', 'condition_text']]
submission.to_csv('submission.csv', index=False)


Fitting 9 folds for each of 108 candidates, totalling 972 fits
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.4s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.4s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.7s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.7s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   1.2s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   1.2s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.4s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.4s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.4s
[CV] END m

In [7]:
submission.isnull().sum()
submission.head(20)

,day_id,condition_text
0,D0001,Partly Cloudy
1,D0002,Partly Cloudy
2,D0003,Light Rain with Thunder
3,D0004,Clear and Sunny
4,D0005,Clear and Sunny
5,D0006,Clear and Sunny
6,D0007,Partly Cloudy
7,D0008,Partly Cloudy
8,D0009,Partly Cloudy
9,D0010,Partly Cloudy
